<a href="https://colab.research.google.com/github/avipreet21/cap-comp215/blob/main/labs/lab3-neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 3 (NEO)
----------------
#### Name: Avipreet singh
#### Date: 2023-01-30

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'  # substitute your API key here

today = str(datetime.date.today())  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
print(data)
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos)

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-01-24&end_date=2023-01-24&detailed=false&api_key=mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T', 'prev': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-01-22&end_date=2023-01-22&detailed=false&api_key=mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T', 'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-01-23&end_date=2023-01-23&detailed=false&api_key=mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'}, 'element_count': 7, 'near_earth_objects': {'2023-01-23': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2496816?api_key=mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'}, 'id': '2496816', 'neo_reference_id': '2496816', 'name': '496816 (1989 UP)', 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2496816', 'absolute_magnitude_h': 20.56, 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2053784995, 'estimated_diameter_max': 0.459240286}, 'meters': {'estimated_diameter_min': 205.3784995184, 'estimate

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

1 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroid')

1 potentially hazardous asteroid


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 20.56,
 'close_approach_data': [{'close_approach_date': '1910-10-01',
                          'close_approach_date_full': '1910-Oct-01 06:55',
                          'epoch_date_close_approach': -1869843900000,
                          'miss_distance': {'astronomical': '0.4234359777',
                                            'kilometers': '63345120.345287499',
                                            'lunar': '164.7165953253',
                                            'miles': '39360832.5965015262'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '71115.3298851714',
                                                'kilometers_per_second': '19.7542583014',
                                                'miles_per_hour': '44188.3003944833'}},
                         {'close_approach_date': '1915-12-29',
                          'close_approach_date_full': '1915-Dec-29 07:1

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
close_approach = [item for item in data['close_approach_data']]

def tuple_creater(close_data):
  new_data = []
  num = 0
  for i in range(len(close_data)):
    a = close_data[i]['close_approach_date']
    b = close_data[i]['miss_distance']['kilometers']
    new_data.append((a,b))
  return new_data

tuple_creater(close_approach)

[('1910-10-01', '63345120.345287499'),
 ('1915-12-29', '12065148.878671504'),
 ('1928-06-17', '10573160.043623126'),
 ('1938-11-18', '777713.040549022'),
 ('1938-11-19', '1037688.003861133'),
 ('1961-10-03', '60219654.610441952'),
 ('1966-12-29', '11555181.184078559'),
 ('1989-11-03', '7401917.315678218'),
 ('1995-01-26', '57534991.131938662'),
 ('2007-06-24', '11990478.971834379'),
 ('2017-11-02', '8218711.812418798'),
 ('2023-01-23', '48695062.76498717'),
 ('2038-07-31', '8226942.388030458'),
 ('2045-10-04', '60397580.796858549'),
 ('2050-12-08', '3998178.673729806'),
 ('2073-10-01', '68855990.897772214'),
 ('2078-12-13', '4871742.031625654'),
 ('2081-11-01', '10572506.585167179'),
 ('2093-12-29', '6971323.662864122'),
 ('2101-10-22', '23825584.055028697'),
 ('2107-01-25', '49096699.012421745'),
 ('2124-09-30', '73483376.761622681'),
 ('2129-12-20', '6349160.752205808'),
 ('2152-12-10', '4232768.494159913'),
 ('2170-10-07', '59725466.588935329'),
 ('2176-01-19', '33311169.199408819')

## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
from datetime import timedelta
start_date = str(datetime.date.today())
end_date = str(datetime.date.today() - timedelta(days = 7))
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text) 

n_results = data['element_count']
neos = data['near_earth_objects']
print(f'{n_results} Near Earth Objects found from{start_date} to {end_date}')
pprint(neos)



70 Near Earth Objects found from2023-01-23 to 2023-01-16
{'2023-01-16': [{'absolute_magnitude_h': 21.3,
                 'close_approach_data': [{'close_approach_date': '2023-01-16',
                                          'close_approach_date_full': '2023-Jan-16 '
                                                                      '19:25',
                                          'epoch_date_close_approach': 1673897100000,
                                          'miss_distance': {'astronomical': '0.3770382016',
                                                            'kilometers': '56404111.867990592',
                                                            'lunar': '146.6678604224',
                                                            'miles': '35047889.9225184896'},
                                          'orbiting_body': 'Earth',
                                          'relative_velocity': {'kilometers_per_hour': '83038.5972488146',
                        

In [ ]:
def new_tuple(whole_data):
  num = 7
  new_data = []
  for itme in range(len(whole_data)):
    start_point = datetime.date.today() - timedelta(days = num)
    for item in range(len(whole_data[str(start_point)])):
      a = whole_data[str(start_point)][item]['id']
      b = whole_data[str(start_point)][item]['close_approach_data'][0]['miss_distance']['kilometers']
      new_data.append((a,start_point,b))
    num = num - 1
  return new_data
result = new_tuple(neos)
result_dist = [float(result[item][2]) for item in range(len(result))]
result_dist.sort()
min_value = str(result_dist.pop(0))

result_closest = [item for item in result if item[2] == min_value]

print(f'Neo with ID: {result_closest[0][0]} passes this {result_closest[0][2]}km close from earth on {result_closest[0][1]}.')

Neo with ID: 3986732 passes this 1993300.692164614km close from earth on 2023-01-20.


# LAB3
## Exercise 1: Define an Asteroid class
Using what we learned in the textbook, define a simple Asteroid class with some basic state 
variables for a single NEO. Your Asteroid class should define at least 4 "state variables:”
 id, name, estimated_diameter (m), is_potentially_hazardous (bool)
 Provide an __init__(self, ...) method to initialize a new Asteroid object with 
specific data values and __str__(self), __repr__(self) methods that returns a 
nicely formatted string representation of the object...
OR... use a @dataclass and allow it to supply all that boilerplate code!
 Provide a little code to test your new class.

## Exercise 2: Factory method: Asteriod.from_NEO
We want to be able to construct Asteroid objects easily from the NEO API. Define a 
“@classmethod”: from_NEO(cls, neo_id) that takes the id for a single NEO, fetches the 
NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is sometimes called a “Factory” – it constructs an object from raw materials

In [ ]:
from dataclasses import dataclass
@dataclass
class Astroid:
  id : str
  name : str
  estimated_diameter : str
  is_potentially_hazardous : bool
  close_approach_list : list

  def __str__(self):
    return f'An astroid id: {self.id} whose name is {self.name} with estimated diamter {self.estimated_diameter} is potentially a hazadous {self.is_potentially_hazardous}'
  
  @classmethod
  def from_NEO(cls, neo_id):
    API_KEY = 'mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    id = neo_id
    name = data['name']
    estimated_diameter = data['estimated_diameter']['kilometers']['estimated_diameter_max']
    is_poten_hazard = data['is_potentially_hazardous_asteroid']
    close_approach_list = []
    return cls(id ,name,estimated_diameter,is_poten_hazard, close_approach_list )
    
  @classmethod
  def rest_data(cls, Asteroid):
    API_KEY = 'mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{Asteroid.id}/?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    close_data = data['close_approach_data']
    close_approach_list = [CloseApproach.single_approach(Asteroid,num_data) for num_data in range(len(close_data))]
    return cls(Asteroid.id , Asteroid.name, Asteroid.estimated_diameter, Asteroid.is_potentially_hazardous, close_approach_list)

my_asteroid = Astroid.from_NEO('2002212')
my_asteroid

Astroid(id='2002212', name='2212 Hephaistos (1978 SB)', estimated_diameter=11.7500571456, is_potentially_hazardous=False, close_approach_list=[])

## Exercise 3: Define a CloseApproach class

---




Each NEO comes with a list of “close_approach_data”, where each record in this list represents 
a single “close approach” to another orbiting body. 
 Develop a class named “CloseApproach” to represent a single close approach record. 
State variables are asteroid (Asteroid object), orbiting body (str), approach date
(datetime object!), miss distance (float choose units, document it, and be consistent!), 
and relative velocity (ditto).
 Define a Factory class method to construct a CloseApproach object from one close 
approach data record (a dictionary object). This method also takes an Asteroid object as 
input – the Asteroid to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [ ]:
@dataclass
class CloseApproach:
  Asteroid : object
  orbiting_body : str
  approach_date : object
  miss_distance : float
  relative_velocity : float

  @classmethod
  def single_approach(cls, Asteroid,i):
    API_KEY = 'mtftH4scwMOfFBB6A1GYC98gjhDNcnwr03SuBA8T'
    neo_id = Asteroid.id
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    close_data = data['close_approach_data']
    orbiting_body = close_data[i]['orbiting_body']
    approach_date = close_data[i]['close_approach_date']
    miss_distance = close_data[i]['miss_distance']['kilometers']
    relative_velocity = close_data[i]['relative_velocity']['kilometers_per_hour']
    return cls(Asteroid.id ,orbiting_body,approach_date,miss_distance,relative_velocity)

closes_my_asteroid = CloseApproach.single_approach(my_asteroid,1)
closes_my_asteroid

CloseApproach(Asteroid='2002212', orbiting_body='Merc', approach_date='1908-06-14', miss_distance='13269177.641424528', relative_velocity='98363.609079358')

## Exercise 4: Add list of CloseApproaches to Asteroid
Every Asteroid should have a list of “close approaches”. Add a new state variable to your Asteroid 
class, initially an empty list. In the factory method, use a list comprehension to build a list of 
CloseApproach objects for the Asteroid from the NEO data record.
Extend your test code to demonstrate this new feature.

In [ ]:
rest = Astroid.rest_data( my_asteroid)
rest

Astroid(id='2002212', name='2212 Hephaistos (1978 SB)', estimated_diameter=11.7500571456, is_potentially_hazardous=False, close_approach_list=[CloseApproach(Asteroid='2002212', orbiting_body='Venus', approach_date='1905-03-14', miss_distance='17637393.49818178', relative_velocity='105916.1280772568'), CloseApproach(Asteroid='2002212', orbiting_body='Merc', approach_date='1908-06-14', miss_distance='13269177.641424528', relative_velocity='98363.609079358'), CloseApproach(Asteroid='2002212', orbiting_body='Earth', approach_date='1911-10-04', miss_distance='68703720.387025729', relative_velocity='152450.6036794967'), CloseApproach(Asteroid='2002212', orbiting_body='Earth', approach_date='1921-02-12', miss_distance='55735946.071449217', relative_velocity='72358.713666265'), CloseApproach(Asteroid='2002212', orbiting_body='Merc', approach_date='1921-03-21', miss_distance='13935730.593409859', relative_velocity='99577.8354931337'), CloseApproach(Asteroid='2002212', orbiting_body='Venus', app